In [6]:
import comet_ml
import logging

logging.basicConfig(level=logging.INFO)
LOGGER = logging.getLogger("comet_ml")

from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import to_categorical

def build_model_graph(experiment):
    model = Sequential()
    model.add(
        Dense(
            experiment.get_parameter("first_layer_units"),
            activation="sigmoid",
            input_shape=(784,),
        )
    )
    model.add(Dense(128, activation="sigmoid"))
    model.add(Dense(128, activation="sigmoid"))
    model.add(Dense(10, activation="softmax"))
    model.compile(
        loss="categorical_crossentropy",
        optimizer=RMSprop(),
        metrics=["accuracy"],
    )
    return model

def train(experiment, model, x_train, y_train, x_test, y_test):
    model.fit(
        x_train,
        y_train,
        batch_size=experiment.get_parameter("batch_size"),
        epochs=experiment.get_parameter("epochs"),
        validation_data=(x_test, y_test),
    )

def evaluate(experiment, model, x_test, y_test):
    score = model.evaluate(x_test, y_test, verbose=0)
    LOGGER.info("Score %s", score)

import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GroupShuffleSplit
from sklearn.svm import SVC
from skopt import BayesSearchCV

SEGMENTS_LENGTH = 3
EXPERIMENT_ID = F'BayesianSearch_{SEGMENTS_LENGTH}s'

data_dir = 'D:\FINKI\8_dps\Project\DATA\MODELS_DATA\\base_models_all_dft_coefficients'
x_train = pd.read_csv(f'{data_dir}\\segments_{SEGMENTS_LENGTH}s_train.csv')
x_test = pd.read_csv(f'{data_dir}\\segments_{SEGMENTS_LENGTH}s_test.csv')

y_train = x_train.pop('episode')
y_test = x_test.pop('episode')

# The optimization config:
config = {
    "algorithm": "bayes",
    "name": "Optimize MNIST Network",
    "spec": {"maxCombo": 10, "objective": "minimize", "metric": "loss"},
    "parameters": {
        "first_layer_units": {
            "type": "integer",
            "mu": 500,
            "sigma": 50,
            "scalingType": "normal",
        },
        "batch_size": {"type": "discrete", "values": [64, 128, 256]},
    },
    "trials": 1,
    # 'api_key': "A8Lg71j9LtIrsv0deBA0DVGcR",
    # 'workspace': "8_dps",
    # 'project_name': "initial",
}

opt = comet_ml.Optimizer(config)

# comet_ml_project_name = 'af-vs-noaf-models'
#         experiment = comet_ml.Experiment(
#
#             project_name = comet_ml_project_name,
#             workspace = "annsegments",
#             auto_output_logging = 'native',
#             auto_weight_logging = True,
#             auto_metric_logging = True,
#             auto_param_logging = True,
#             auto_histogram_tensorboard_logging = True,
#             auto_histogram_weight_logging = True,
#             auto_histogram_gradient_logging = True,
#             auto_histogram_activation_logging = True
#             # offline_directory = f'{PROJECT_DIR}\_COMETML\\{comet_ml_project_name}_{datetime.now().strftime("%Y%m%d-%Hh%Mm")}'
#         )

for experiment in opt.get_experiments():
    # Log parameters, or others:
    experiment.log_parameter("epochs", 10)

    # Build the model:
    model = build_model_graph(experiment)

    # Train it:
    train(experiment, model, x_train, y_train, x_test, y_test)

    # How well did it do?
    evaluate(experiment, model, x_test, y_test)

    # Optionally, end the experiment:
    experiment.end()

ValueError: Invalid COMET_API_KEY